# Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [2]:
import os

In [3]:
print(os.getcwd())

c:\Users\Javi\Desktop\CSE 120\Anomaly Detection\Leaf-Classification\data\tomato


In [4]:
train_data_dir = 'train'

# Check if the directory exists
if not os.path.exists(train_data_dir):
    print(f"The directory {train_data_dir} does not exist.")
else:
    print(f"Found directory {train_data_dir}. Proceeding with loading data...")
    # Proceed with your data loading and model training


Found directory train. Proceeding with loading data...


# Data loading and preprocessing

In [5]:
train_data_dir = 'train'
test_data_dir = 'val'

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3
)

In [7]:
train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=128, # increased from 32 to 128
    class_mode='categorical',
    subset='training'
)

Found 7000 images belonging to 10 classes.


In [8]:
val_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=128, # increased from 32 to 128
    class_mode='categorical',
    subset='validation'
)

Found 3000 images belonging to 10 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1/255.0)

In [10]:
test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=128, # increased from 32 to 128
    class_mode='categorical',
    shuffle=False
)

Found 1000 images belonging to 10 classes.


# Model architecture

In [11]:
model = Sequential([
    Input(shape=(224, 224, 3)),  # Define the input shape explicitly here
    #Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Assuming we have 10 classes, adjust if necessary
])

# Compile the model

In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.0001), # add learning rate (lr=0.0001) default was 0.001
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early Stopping to avoid overfitting

In [13]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    verbose=1,
    restore_best_weights=True
)

# Train the model

In [14]:
history = model.fit(
    train_gen,
    epochs=160, #increased from 80 to 160
    validation_data=val_gen,
    callbacks=[early_stopping]
)

Epoch 1/160
55/55 [==============================] - 83s 2s/step - loss: 2.2986 - accuracy: 0.1101 - val_loss: 2.2929 - val_accuracy: 0.1160
Epoch 2/160
55/55 [==============================] - 79s 1s/step - loss: 2.2831 - accuracy: 0.1403 - val_loss: 2.2679 - val_accuracy: 0.1467
Epoch 3/160
55/55 [==============================] - 79s 1s/step - loss: 2.2385 - accuracy: 0.1816 - val_loss: 2.2020 - val_accuracy: 0.1813
Epoch 4/160
55/55 [==============================] - 78s 1s/step - loss: 2.1486 - accuracy: 0.2259 - val_loss: 2.1029 - val_accuracy: 0.2623
Epoch 5/160
55/55 [==============================] - 79s 1s/step - loss: 2.0437 - accuracy: 0.2503 - val_loss: 1.9947 - val_accuracy: 0.2660
Epoch 6/160
55/55 [==============================] - 79s 1s/step - loss: 1.9497 - accuracy: 0.2926 - val_loss: 1.9297 - val_accuracy: 0.3083
Epoch 7/160
55/55 [==============================] - 79s 1s/step - loss: 1.8787 - accuracy: 0.3353 - val_loss: 1.8727 - val_accuracy: 0.3043
Epoch 8/160
5


# Evaluate the model

In [15]:
scores = model.evaluate(test_gen) #change test_gen to val_gen
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

8/8 [==============================] - 2s 254ms/step - loss: 1.7777 - accuracy: 0.5720
Test loss: 1.7777217626571655
Test accuracy: 0.5720000267028809


# Save the model

In [16]:
model.save('merged_model.h5')